#### Activities (aggregated last 30 days)

##### Data ingestion strategy:
<mark style="background: #88D5FF;">**REPLACE**</mark>

##### Related pipeline:

**Load_Activities_E2E**

##### Source:

**1 Delta table**  from FUAM_Lakehouse
- **activities**

##### Target:

**1 Delta table** in FUAM_Lakehouse 
- **aggr_table_name** variable value

In [6]:
## Parameters
display_data = False

In [7]:
## Variables
aggr_table_name = "aggregated_activities_last_30days"

In [8]:
# Get aggregated Gold data
query = """
SELECT 
    CreationDate,
    CreationDateKey,
    CreationHour,
    Activity,
    Experience,
    Workload,
    UPPER(CapacityId) as CapacityId,
    UPPER(WorkspaceId) as WorkspaceId,
    GatewayId,
    COALESCE(UPPER(ArtifactId), UPPER(ObjectId)) AS fuam_item_id,
    COALESCE(ObjectType, ArtifactKind) AS fuam_item_kind,
    CASE 
        WHEN ResultStatus = 'Failed' THEN 'Failed' 
        WHEN ResultStatus = 'InProgress' THEN 'InProgress'
        WHEN ResultStatus = 'Succeeded' THEN 'Succeeded'
        WHEN IsSuccess = 1 THEN 'Succeeded'
        ELSE 'N/A'
    END AS fuam_activity_status,
    COUNT(*) AS CountOfActivities,
    COUNT(DISTINCT UserId) AS DistinctCountOfUsers
FROM FUAM_Lakehouse.activities
WHERE CreationDate >= date_sub(CAST(current_timestamp() as DATE), 30)
GROUP BY
    CreationDate,
    CreationDateKey,
    CreationHour,
    Activity,
    Experience,
    Workload,
    CapacityId,
    WorkspaceId,
    GatewayId,
    ArtifactId,
    ItemId,
    ObjectId,
    ObjectType,
    ArtifactKind,
    ResultStatus,
    IsSuccess
ORDER BY CreationDate DESC
"""

# Query data
agg_df = spark.sql(query) 



In [9]:
if display_data:
    display(agg_df)

In [10]:
# Overwrite aggregated table
agg_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable(aggr_table_name)